In [8]:
import tensorflow as tf
from IPython.display import display
import pandas as pd
import seaborn as sns
import os
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [10]:
from helper import *

In [9]:
image_ext = '.jpeg'
data_dir = 'dataset/'
test_dir = data_dir+'test/'
train_dir = data_dir+'train/'
val_dir = data_dir+'val/'
normals = 'NORMAL/'
infected = 'PNEUMONIA/'
training_normal_nos = len(glob.glob1(train_dir+normals,'*'+image_ext))
training_infected_nos = len(glob.glob1(train_dir+infected,'*'+image_ext))
training_nos = training_infected_nos+training_normal_nos
testing_normal_nos = len(glob.glob1(test_dir+normals,'*'+image_ext))
testing_infected_nos = len(glob.glob1(test_dir+infected,'*'+image_ext))
testing_nos = testing_infected_nos+testing_normal_nos
validation_normal_nos = len(glob.glob1(val_dir+normals,'*'+image_ext))
validation_infected_nos = len(glob.glob1(val_dir+infected,'*'+image_ext))
validation_nos = validation_infected_nos+validation_normal_nos

print('Total number of training images: ', training_nos)
print('Total number of testing images: ', testing_nos)
print('Total number of validation images: ', validation_nos)

Total number of training images:  5216
Total number of testing images:  624
Total number of validation images:  16


In [11]:
rescale_factor = 1./255
target_size = (150,150)
class_mode = 'categorical'
training_batch_size = int(training_nos/40)

train_data_generator = create_image_data_generator(target_size=target_size, 
                                                   rescale=rescale_factor, 
                                                   path=train_dir, 
                                                   batch_size=training_batch_size)

test_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=test_dir,
                                                 batch_size=testing_nos)

val_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=val_dir,
                                                 batch_size=validation_nos)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
model_dir = 'models/'
log_dir = 'logs/'
